In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='2'
#import wget
import sys
sys.path.append('../')
import argparse
import json
import pandas as pd
import random
import numpy as np
import string
import nltk
from functools import partial
import re
from  tqdm import tqdm
import torch
nltk.download('punkt')
from src.dataset_classes import DatasetObject,Features,SquadQuestionGenerationDataset
from src.utils import answerGeneratorDataset,questionGeneratorDataset,buildFact,setuptokenizer,pad_seq,SmartCollator
from dataclasses import dataclass, field
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
model_base = 'facebook/bart-base'
tokenizer = setuptokenizer(model_base=model_base,
                           special_tokens=['<section>','</section>'
                                           ,'<generate_questions>',
                                           '<generate_answers>'])

In [4]:
from src.model_utils import model_init
# ../trained_models/t5_base_model_1/checkpoint-18000/pytorch_model.bin
trained_weights = torch.load('../trained_models/bart_base_model_1/checkpoint-6000/pytorch_model.bin')

generator = model_init(model_base=model_base,vocab_size=len(tokenizer))
generator.load_state_dict(trained_weights)
device = generator.device

In [5]:
generator.load_state_dict(trained_weights)

<All keys matched successfully>

In [6]:
from transformers import TrainingArguments,Trainer
train_args = TrainingArguments(overwrite_output_dir=True, output_dir='trained_models/setup_1/',
                               evaluation_strategy='steps',
                               lr_scheduler_type='cosine',
                               adafactor=False,
                               load_best_model_at_end=True,
                               save_total_limit=1,
                               weight_decay=0.3,
                               warmup_ratio=0.35,
                               num_train_epochs=5,
                               per_device_train_batch_size=16,
                               push_to_hub=True,
                               hub_model_id="kaejo98/bart-base_question_generation",
                               )


In [7]:
trainer = Trainer(generator,train_args,tokenizer=tokenizer)

Cloning https://huggingface.co/kaejo98/bart-base_question_generation into local empty directory.


In [8]:
trainer.push_to_hub("New questions generation")

Saving model checkpoint to trained_models/setup_1/
Configuration saved in trained_models/setup_1/config.json
Model weights saved in trained_models/setup_1/pytorch_model.bin
tokenizer config file saved in trained_models/setup_1/tokenizer_config.json
Special tokens file saved in trained_models/setup_1/special_tokens_map.json
added tokens file saved in trained_models/setup_1/added_tokens.json


Upload file pytorch_model.bin:   0%|          | 32.0k/532M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.37k/3.37k [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/kaejo98/bart-base_question_generation
   7995090..9bc25ab  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}}
To https://huggingface.co/kaejo98/bart-base_question_generation
   9bc25ab..658cc99  main -> main



'https://huggingface.co/kaejo98/bart-base_question_generation/commit/9bc25ab53ff6766f2b2554d0923bda2411228855'

In [1]:
from huggingface_hub import notebook_login

In [3]:
notebook_login()

Login successful
Your token has been saved to /home/nlplab/.huggingface/token
